# Machine Failure Prediction using ML Models

This notebook performs machine failure prediction using Random Forest, SVM, and XGBoost, starting from your cleaned dataset.

In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split


# Load the dataset
data = pd.read_csv("predictive_maintenance.csv")
np.random.seed(0)

# 1. Drop UDI because it is not used and destroys model efficiency -------------------------------------
data.drop(columns=['UDI'], inplace=True)

# 2. Check for Missing Values ----------------------------------------
missing_values = data.isnull().sum()
if missing_values.sum() == 0:
    print("No missing values found.")
else:
    print(" Missing values detected:\n", missing_values)

# 3. Encode Categorical Features -------------------------------------
label_enc = LabelEncoder()

# Encoding Product ID
data['Product ID'] = label_enc.fit_transform(data['Product ID'])

# Encoding Type and Failure Type
data['Type'] = label_enc.fit_transform(data['Type'])
data['Failure Type'] = label_enc.fit_transform(data['Failure Type'])


# 5. Final Data Check -----------------------------------------------
print("\n🔍 Final Data Check:")
print(data.info())

# Save cleaned data
data.to_csv("cleaned_data.csv", index=False)
print("\n Cleaned data saved as 'cleaned_data.csv'.")

scaler = StandardScaler()


# Define features and target
X = data.drop(columns=['Target'])  # Features
y = data['Target']                 # Target label (0 = No Failure, 1 = Failure)

# Scale the features for models that are sensitive to feature scales (like SVM)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and test sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


No missing values found.

🔍 Final Data Check:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product ID               10000 non-null  int64  
 1   Type                     10000 non-null  int64  
 2   Air temperature [K]      10000 non-null  float64
 3   Process temperature [K]  10000 non-null  float64
 4   Rotational speed [rpm]   10000 non-null  int64  
 5   Torque [Nm]              10000 non-null  float64
 6   Tool wear [min]          10000 non-null  int64  
 7   Target                   10000 non-null  int64  
 8   Failure Type             10000 non-null  int64  
dtypes: float64(3), int64(6)
memory usage: 703.3 KB
None

 Cleaned data saved as 'cleaned_data.csv'.


In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Initialize and train the models

# Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Support Vector Machine
svm = SVC(kernel='rbf', probability=True)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

# XGBoost
xgb = XGBClassifier(eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)


In [37]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluate models using accuracy, precision, recall, and F1-score

# Create a function to extract model performance
def extract_performance(y_test, y_pred, model_name):
    report = classification_report(y_test, y_pred, output_dict=True)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": report["1"]["precision"],
        "Recall": report["1"]["recall"],
        "F1-Score": report["1"]["f1-score"]
    }

# Compile performance metrics
results = {
    "Random Forest": extract_performance(y_test, y_pred_rf, "Random Forest"),
    "SVM": extract_performance(y_test, y_pred_svm, "SVM"),
    "XGBoost": extract_performance(y_test, y_pred_xgb, "XGBoost")
}

# Display results
pd.DataFrame(results).T


,Accuracy,Precision,Recall,F1-Score
Random Forest,0.9980,0.967213,0.967213,0.967213
SVM,0.9990,1.000000,0.967213,0.983333
XGBoost,0.9985,0.983333,0.967213,0.975207
